In [ ]:
import pandas as pd
import subprocess

In [ ]:
def get_the_table_data():
    while True:
        table_path = input("What is the table path + name? (.../filename.csv):")
        try:
            video_schnitt_df = pd.read_csv(table_path)
            print("Table loaded succesfully")
            return video_schnitt_df
        except Exception as error:
            print("An error occured:", error)
            continue
        break

In [ ]:
def clean_the_data(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "_")
    df.columns = df.columns.str.strip()
    print("Table Columns cleaned")
    return df

In [ ]:
def cut_head_tail(original_video,output_file,cut_head="00:00:00",cut_tail="00:50:00"):

    # Construct the command
    command = [
        'ffmpeg',
        '-i', original_video,
        '-ss', cut_head,
        '-to', cut_tail,
        '-c', 'copy',
        output_file
    ]
    
     # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def get_video_duration(file_path):
    command = ['ffmpeg', '-i', file_path]
    result = subprocess.run(command, text=True, capture_output=True)
    output_lines = result.stderr.split('\n')
    duration_line = [line for line in output_lines if 'Duration' in line][0]
    duration = duration_line.strip().split(",")[0].split(" ")[1]
    return duration

In [ ]:
def cut_in_pieces(input_file, timestamp, head_output, tail_output):
    head_cmd = ['ffmpeg', '-i', input_file, '-to', str(timestamp), '-c', 'copy', head_output]
    tail_cmd = ['ffmpeg', '-i', input_file, '-ss', str(timestamp), '-c', 'copy', tail_output]

# Execute the commands
    try:
        subprocess.run(head_cmd, check=True)
        subprocess.run(tail_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

In [ ]:
def get_standbild(input_file,output_file,cut_head):
    
    #creat timedelta and add 1 miliseconds
    time_delta = pd.to_timedelta(cut_head)
    new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

    #create dummy for calculation to transform data type back to datetype
    dummy_date = pd.Timestamp('1900-01-01')
    new_timestamp = dummy_date + new_time_delta

    #get the new time back to wished str format
    cut_tail = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

    # Construct the command
    command = [
        'ffmpeg',
        '-i', input_file,
        '-ss', cut_head,
        '-to', cut_tail,
        '-an', #audio no
        '-c:v', 'copy', output_file
    ]

    # Run the command
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

# Get the file and clean it

In [ ]:
video_schnitt_df = get_the_table_data()
video_schnitt_df = clean_the_data(video_schnitt_df)

# Cut head and tail

In [ ]:
# videos need to be in same directory with python script
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split(".")[0]}_ohne_start_ende.mp4"
        cut_head = video_schnitt_df["vorne_abschneiden_bis"][idx]
        cut_tail = video_schnitt_df["hinten_abschneiden_ab"][idx]

        #building logic when just to cut head or tail
        if str(cut_head) == "nan":
            cut_head = "00:00:00"
        if str(cut_tail) == "nan":
            cut_tail = "00:50:00"
        
        cut_head_tail(video_name, output_file, cut_head, cut_tail)
except Exception as error:
    print("An error occurred:", error)

# Get Standbild

In [ ]:
# videos need to be in same directory with python script
try:
    for idx, video_name in video_schnitt_df["dateiname"].items():
        output_file = f"{video_name.split(".")[0]}_standbild.mp4"
        cut_head = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

        #building logic when just to cut head or tail
        if str(cut_head) == "nan":
            continue
        
        get_standbild(video_name,output_file,cut_head)
except Exception as error:
    print("An error occurred:", error)

In [ ]:
def get_audio(input_file,output_file,cut_head,cut_tail):
    command = ['ffmpeg','-i', input_file,'-ss', cut_head,'-to', cut_tail,'-vn','-c:a', 'copy', output_file]
    try:
        subprocess.run(command, check=True)
    except:
        continue
# create function to get audio from start till cut_point 
# cut_head = "Vorne Bild durch Standbild ersetzen bis" - "Vorne abschneiden bis" 
# cut_tail =  "Vorne Bild durch Standbild ersetzen bis"
# use subprocess and ffmpeg to cut out the audio
# just audio no video! 
# new_name = ...start_audio.mp4


# Cut in pieces

In [ ]:
for idx, video_name in video_schnitt_df["dateiname"].items():
    cut_time = video_schnitt_df["schnitt_setzen_bei"][idx]
    head_output = f"{video_name}_head_piece.mp4"
    tail_output = f"{video_name}_tail_piece.mp4"
    cut_in_pieces(video_name,cut_time,head_output,tail_output)

In [ ]:
# videos need to be in same directory with python script
for idx, video_name in video_schnitt_df["dateiname"].items():
    output_file = f"{video_name}_standbild.mp4"
    cut_time = video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][idx]

In [ ]:

# Your time string
time_str = "00:00:00.000"

# Convert to Pandas Timedelta
time_delta = pd.to_timedelta(time_str)

# Add one millisecond
new_time_delta = time_delta + pd.Timedelta(milliseconds=1)

# Convert Timedelta to Timestamp for formatting (using a dummy date)
dummy_date = pd.Timestamp('1900-01-01')
new_timestamp = dummy_date + new_time_delta

# Format to "HH:MM:SS.fff"
new_time_str = new_timestamp.strftime('%H:%M:%S.%f')[:-3]

new_time_str


In [ ]:
video_schnitt_df["vorne_bild_durch_standbild_ersetzen_bis"][0]


In [ ]:
get_standbild("Median.mp4","standbild.mp4","00:05:00","00:05:01")

In [ ]:
duration = pd.to_datetime(get_video_duration("Video.mp4")).strftime('%H:%M:%S.%f')[:-3]
duration = duration + duration

In [ ]:
start_time = pd.to_datetime("00:00:10").strftime('%H:%M:%S.%f')[:-3]
end_time = pd.to_datetime("00:00:15").strftime('%H:%M:%S.%f')[:-3]

In [ ]:
start_time = pd.to_datetime("00:00:10").strftime('%H:%M:%S.%f')[:-3]
end_time = pd.to_datetime("00:00:15").strftime('%H:%M:%S.%f')[:-3]
result = start_time + pd.to_timedelta(end_time) 

In [ ]:
result

In [ ]:
get_video_duration("Video.mp4")